## Capstone Project

### Modeling Notebook - `DecisionTrees`  `COMBINED DATASET TREE` from 10/24/19

#### Importing Libraries

In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn import metrics

# scikit-learn libraries for feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFECV

# scikit-learn libraries for learning
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

/Users/nate_velarde/anaconda/envs/py3jp/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Directory/File Structure

In [2]:
sys.version

'3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
print ('Running pandas version:', pd.__version__)
print ('Running numpy version:', np.__version__)
print ('Running sklearn version:', sklearn.__version__)

Running pandas version: 0.25.1
Running numpy version: 1.14.2
Running sklearn version: 0.21.3


In [4]:
os.getcwd()

'/Users/nate_velarde/Documents/UC_Berkeley/Courses/W210_Capstone/stroke_project/sandbox/notebooks'

In [5]:
os.chdir('../data')

In [6]:
sorted(os.listdir())

['.DS_Store',
 '273_vs_281_null_count_by_feature.csv',
 '273_vs_281_null_count_by_feature.xlsm',
 'Capstone - Complication list - complete.xlsx',
 'Capstone - STS risk factor list.xlsx',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'PREOP_dataset_10_24.pkl',
 'PREOP_dataset_TREE_10_24.pkl',
 'X_A_DREF.pkl',
 'X_A_DREF_TREE_SKLEARN.pkl',
 'X_PREOP_10_24.pkl',
 'X_PREOP_TREE_10_24.pkl',
 'X_dev_A_DREF.pkl',
 'X_dev_A_DREF_TREE_SKLEARN.pkl',
 'X_dev_PREOP_10_24.pkl',
 'X_dev_PREOP_TREE_10_24.pkl',
 'X_test_A_DREF.pkl',
 'X_test_A_DREF_TREE_SKLEARN.pkl',
 'X_test_PREOP_10_24.pkl',
 'X_test_PREOP_TREE_10_24.pkl',
 'X_train_A_DREF.pkl',
 'X_train_A_DREF_TREE_SKLEARN.pkl',
 'X_train_PREOP_10_24.pkl',
 'X_train_PREOP_TREE_10_24.pkl',
 'capstone_STS_risk_factor_features.xlsx',
 'capstone_cleaned_data.csv',
 'capstone_data-version-2.xlsx',
 'capstone_data.xlsx',
 'capstone_data_binarized_outcome.pkl',
 'capstone_data_binarized_outcome.xlsx',
 'capstone_data_binarized_outcome_compressed.pkl',


#### Loading Datasets

#### `X_train`, `y_train`
- designation of `_all` denotes complete feature set

In [7]:
X_train_all = pd.read_pickle('X_train_PREOP_TREE_10_24.pkl')
y_train = pd.read_pickle('y_train_PREOP_TREE_10_24.pkl')

In [8]:
X_train_all.shape, y_train.shape

((34192, 73), (34192,))

#### `X_dev`, `y_dev`
- designation of `_all` denotes complete feature set

In [9]:
X_dev_all = pd.read_pickle('X_dev_PREOP_TREE_10_24.pkl')
y_dev = pd.read_pickle('y_dev_PREOP_TREE_10_24.pkl')

In [10]:
X_dev_all.shape, y_dev.shape

((4274, 73), (4274,))

#### `X_test`, `y_test`
- designation of `_all` denotes complete feature set

In [11]:
X_test_all = pd.read_pickle('X_dev_PREOP_TREE_10_24.pkl')
y_test = pd.read_pickle('y_dev_PREOP_TREE_10_24.pkl')

In [12]:
X_test_all.shape, y_dev.shape

((4274, 73), (4274,))

- validating row count for `COMBINED DATASET TREE` from 10/24/19 - `42,740` total observations

In [13]:
42740 - X_train_all.shape[0] - X_dev_all.shape[0] - X_test_all.shape[0]

0

- last look at the data (`X_train_all`) before modeling

In [14]:
X_train_all.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten,cvdstenrt,cvdstenlft,ArrhythDur_when_Combo
0,-2.12457,0.11594,0.32942,0.15700,-2.65268,-0.35221,-2.27155,-1.62927,2.47597,0.98148,3.58585,10,2,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,2,1,1,0,0,0,0,0,0
1,1.08243,-0.82771,-0.95497,-0.39563,-0.10617,-0.13691,0.04597,0.25206,-0.29493,0.33103,0.49141,10,0,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,2,1,2,1,0,0,0,0,0,0
2,-0.20037,1.53140,1.56497,0.35976,0.59504,-0.25533,-0.34028,0.32442,-0.67978,0.57495,-0.12748,10,1,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,1,0,0,0,0,0,0,0,0,1
3,0.44103,-0.31888,-0.21754,-0.04732,0.59504,-0.25533,0.04597,-0.76096,-0.29493,0.16842,-0.64322,11,3,3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,1,0,2,2,0,0,0,0,0,1
4,-0.75014,-1.05900,0.19268,0.56118,-2.17290,-0.36298,0.04597,0.39678,-0.29493,0.57495,-0.65353,6,4,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,2,0,1,1,0,1,2,0,2,1


In [15]:
X_train_all.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten,cvdstenrt,cvdstenlft,ArrhythDur_when_Combo
34187,-0.29200,0.58776,0.78848,0.26686,0.04145,-0.25533,0.04597,0.83093,-0.29493,0.00581,0.56361,1,3,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,1.0,2,0,1,2,0,1,0,0,0,1
34188,-1.11666,0.60626,-0.14917,-0.31935,-0.51214,-0.47063,-2.07842,0.61386,0.57972,-3.16512,1.62604,7,4,3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,2,2,1,2,0,1,0,0,0,0
34189,0.62428,-1.05900,0.17314,0.54625,-0.32761,-0.07232,-0.34028,-1.05040,-0.45237,1.38800,-0.12748,3,0,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,1,2,1,1,0,0,1,3,0,0
34190,0.99080,-1.52157,0.75918,1.25109,-0.19844,-0.47063,0.62535,1.04801,-0.67978,0.57495,2.03863,10,3,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0.0,0,0,2,3,0,0,0,0,0,0
34191,-0.20037,0.60626,0.71034,0.21143,-0.69667,-0.04003,-0.14715,3.14642,-0.36491,0.81886,-0.95266,12,1,3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,2,0,1,0,0,1,0,0,0,1


## Random Forest Classifier
- see Albon 14.4 pages 238-240

- instantiating a `RandomForestClassifier()` object

In [16]:
rf_clf = RandomForestClassifier(class_weight='balanced',
                                random_state=0,
                                n_jobs=-1,
                                verbose=2)

#### Default `RandomForestClassifier()` Settings
- `n_estimators`: `10` -- number of trees in the forest
- `criterion`: `gini` -- function to measure the quality of split -- could also set for `entropy` 
- `max_depth`: `None` -- the maximum depth of the tree -- if `None`, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples 
- `max_features`: `Auto` -- `max_features=sqrt(n_features)` -- can put in a `float` to represent the number of features used -- per `Muller and Guido` page 88, increasing `max_features` might sometimes improve performance
- `min_samples_split`: `2` -- minimum number of samples required to split an internal node

- running out of the box with default settings only takes a few seconds

- creating a `StratifiedKFold` cross validation

In [17]:
skf = StratifiedKFold(n_splits=5,
                      shuffle=True,
                      random_state=0)

- defining `scoring_metrics`

In [18]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

- conduct `StratifiedKFold` cross-validation

In [19]:
skf_results = cross_validate(rf_clf, # model
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores                     

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.4s finished


In [20]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.98289,0.98289,0.98304,0.98304,0.98304,0.98298
train_accuracy,0.99580,0.99543,0.99565,0.99525,0.99481,0.99539
test_f1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
train_f1,0.85890,0.84472,0.85363,0.83791,0.82025,0.84308
test_f1_macro,0.49569,0.49569,0.49572,0.49572,0.49572,0.49571
train_f1_macro,0.92838,0.92120,0.92571,0.91775,0.90881,0.92037
test_f1_weighted,0.97441,0.97441,0.97463,0.97463,0.97463,0.97454
train_f1_weighted,0.99550,0.99508,0.99534,0.99487,0.99435,0.99503
test_precision,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
train_precision,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


#### Key Takeaways
- Model is clearly overfitting
- Can tune `max_depth` parameter to see if that reduces overfitting - pre-prune trees

#### Varying `max_depth`

- instantiating a `RandomForestClassifier()` object

In [21]:
rf_clf = RandomForestClassifier(max_depth=5,
                                class_weight='balanced',
                                random_state=0,
                                n_jobs=-1,
                                verbose=2)

In [22]:
skf_results = cross_validate(rf_clf, # model
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores                     

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.2s finished


In [23]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.74762,0.76956,0.74130,0.75271,0.78078,0.75839
train_accuracy,0.75330,0.76639,0.75360,0.75993,0.78449,0.76354
test_f1,0.05476,0.05967,0.05350,0.05794,0.05066,0.05531
train_f1,0.08115,0.07952,0.07823,0.08141,0.08817,0.08170
test_f1_macro,0.45457,0.46418,0.45184,0.45781,0.46337,0.45835
train_f1_macro,0.46934,0.47287,0.46801,0.47166,0.48299,0.47297
test_f1_weighted,0.84069,0.85485,0.83666,0.84411,0.86208,0.84768
train_f1_weighted,0.84432,0.85284,0.84451,0.84862,0.86435,0.85093
test_precision,0.02926,0.03207,0.02852,0.03097,0.02734,0.02963
train_precision,0.04332,0.04261,0.04178,0.04354,0.04751,0.04375


#### Key Takeaways
- Varying `max_depth` significantly reduced overfitting
- We can next try increasing `n_estimators`
- Per `Muller and Guido` in `Introduction to Machine Learning with Python` page 88, "for `n_estimators`, larger is always better as averaging over more trees will yield a more robust ensemble by reducing overfitting"

#### Varying `n_estimators`

- instantiating a `RandomForestClassifier()` object

In [24]:
rf_clf = RandomForestClassifier(n_estimators=100,
                                max_depth=5,
                                class_weight='balanced',
                                random_state=0,
                                n_jobs=-1,
                                verbose=2)

In [25]:
skf_results = cross_validate(rf_clf, # model
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores              

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   14.6s finished


In [26]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.79003,0.78871,0.77347,0.79365,0.79014,0.78720
train_accuracy,0.79721,0.79041,0.78464,0.79791,0.79689,0.79341
test_f1,0.06753,0.05863,0.05144,0.06121,0.05778,0.05932
train_f1,0.09878,0.09702,0.09578,0.10055,0.10068,0.09856
test_f1_macro,0.47461,0.46982,0.46141,0.47265,0.46985,0.46967
train_f1_macro,0.49227,0.48923,0.48677,0.49336,0.49310,0.49095
test_f1_weighted,0.86777,0.86693,0.85747,0.87013,0.86794,0.86605
train_f1_weighted,0.87237,0.86811,0.86444,0.87278,0.87214,0.86997
test_precision,0.03654,0.03173,0.02769,0.03317,0.03127,0.03208
train_precision,0.05343,0.05235,0.05158,0.05440,0.05445,0.05324


#### Key Takeaways
- marginal improvement from increasing `n_estimators`, but it something we can tune using `GridSearchCV`
- let's try changing `criterion` to `entropy`

#### Changing `criterion` to `entropy`

- instantiating a `RandomForestClassifier()` object

In [27]:
rf_clf = RandomForestClassifier(n_estimators=100,
                                max_depth=5,
                                criterion='entropy',
                                class_weight='balanced',
                                random_state=0,
                                n_jobs=-1,
                                verbose=2)

In [28]:
skf_results = cross_validate(rf_clf, # model
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores              

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   15.4s finished


In [29]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.79003,0.78637,0.76353,0.78795,0.77815,0.78121
train_accuracy,0.79191,0.78909,0.77952,0.79517,0.78372,0.78788
test_f1,0.07115,0.06883,0.05050,0.05844,0.05718,0.06122
train_f1,0.09651,0.09591,0.09404,0.09760,0.09430,0.09567
test_f1_macro,0.47639,0.47409,0.45772,0.46948,0.46573,0.46868
train_f1_macro,0.48946,0.48827,0.48426,0.49104,0.48575,0.48776
test_f1_weighted,0.86777,0.86548,0.85113,0.86657,0.86042,0.86228
train_f1_weighted,0.86905,0.86728,0.86119,0.87107,0.86386,0.86649
test_precision,0.03849,0.03719,0.02710,0.03160,0.03081,0.03304
train_precision,0.05210,0.05172,0.05056,0.05276,0.05077,0.05158


#### Key Takeaways
- no material change and also took longer to calculate, so will go back to using default of `gini`

### Hyperparameter Tuning with `RandomizedSearchCV`
- there are lots of other parameters to tune, so let's try to narrow down our possibilities
- this took more than 10 minutes to run, so saved model which will be reloaded in subsequent runs

In [30]:
random_grid = {'n_estimators': [100, 150, 200, 250],          # default is `10`
               'max_features': ['auto', 0.25, 0.50],          # default is square root of `n_features`
               'max_depth': [2, 5, 10, 20, None],             # default is `None`
               'min_samples_split': [2, 5, 10]}               # default is `2`

- instantiating the base model to tune

In [31]:
rf_clf = RandomForestClassifier(class_weight='balanced')

- Random search of parameters, using 3 fold cross validation
- Search across 50 different combinations, and use all available cores

In [32]:
rf_random_model = RandomizedSearchCV(estimator=rf_clf, 
                                     param_distributions=random_grid, 
                                     n_iter=100, 
                                     cv=3,
                                     scoring='roc_auc',
                                     verbose=2, 
                                     random_state=42, 
                                  n_jobs=-1)

- fitting the random search model

In [33]:
#rf_random_model.fit(X_train_all, y_train)

- takes 10 minutes to run

#### Saving `rf_random model`

In [34]:
#joblib.dump(rf_random_model, 'rf_randomized_search.sav')

#### Loading saved `rf_random model`

In [35]:
loaded_rf_random_model = joblib.load('rf_randomized_search.sav')

In [36]:
loaded_rf_random_model.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'max_features': 'auto',
 'max_depth': 2}

- examining results

In [37]:
randomized_search_results = pd.DataFrame(loaded_rf_random_model.cv_results_)

In [38]:
randomized_search_results = randomized_search_results.sort_values(by=['mean_test_score'],
                                                                  ascending=False)

In [39]:
display_cols = ['params',
                'mean_test_score']

In [40]:
randomized_search_results_summary = randomized_search_results.copy()[display_cols].reset_index(drop=True)

In [41]:
randomized_search_results_summary.head(20)

,params,mean_test_score
0,"{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 'auto', 'max_depth': 2}",0.66698
1,"{'n_estimators': 150, 'min_samples_split': 10, 'max_features': 'auto', 'max_depth': 2}",0.66630
2,"{'n_estimators': 200, 'min_samples_split': 5, 'max_features': 'auto', 'max_depth': 2}",0.66563
3,"{'n_estimators': 100, 'min_samples_split': 5, 'max_features': 'auto', 'max_depth': 2}",0.66499
4,"{'n_estimators': 250, 'min_samples_split': 10, 'max_features': 'auto', 'max_depth': 2}",0.66461
5,"{'n_estimators': 200, 'min_samples_split': 10, 'max_features': 0.25, 'max_depth': 2}",0.66416
6,"{'n_estimators': 200, 'min_samples_split': 10, 'max_features': 'auto', 'max_depth': 2}",0.66375
7,"{'n_estimators': 200, 'min_samples_split': 10, 'max_features': 'auto', 'max_depth': 5}",0.66328
8,"{'n_estimators': 250, 'min_samples_split': 2, 'max_features': 'auto', 'max_depth': 5}",0.66297
9,"{'n_estimators': 200, 'min_samples_split': 2, 'max_features': 'auto', 'max_depth': 2}",0.66276


#### Key Takeaways
- clear that `max_depth` is the key parameter - dominates the top 10 results
- will run a `GridSearchCV` to confirm

### Tuning `RandomForestClassifier()` Hyperparameters using `GridSearchCV`
- instantiating a `RandomForestClassifier()` object

In [42]:
rf_clf = RandomForestClassifier(class_weight='balanced',
                                random_state=0)

- defining hyperparamter candidates

In [43]:
hyper_params = {'n_estimators': [200],          
                'max_features': ['auto'],          
                'max_depth': [2, 5],             
                'min_samples_split': [2, 5, 10]}               

- create `GridSearchCV` object

In [44]:
gs = GridSearchCV(rf_clf, # estimator
                  hyper_params, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5, 
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

- fit `GridSearchcV`

In [45]:
rf_gs_model = gs.fit(X_train_all,
                     y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.5min finished


- examining results

In [46]:
gs_results = pd.DataFrame(rf_gs_model.cv_results_)

In [47]:
gs_results = gs_results.sort_values(by=['mean_test_roc_auc'],
                                           ascending=False)

In [48]:
display_cols = ['params',
                'rank_test_roc_auc',
                'mean_test_roc_auc',
                'mean_train_roc_auc']

In [49]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [50]:
gs_results_summary

,params,rank_test_roc_auc,mean_test_roc_auc,mean_train_roc_auc
0,"{'max_depth': 2, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 200}",1,0.66622,0.70836
1,"{'max_depth': 2, 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 200}",1,0.66622,0.70836
2,"{'max_depth': 2, 'max_features': 'auto', 'min_samples_split': 10, 'n_estimators': 200}",1,0.66622,0.70836
3,"{'max_depth': 5, 'max_features': 'auto', 'min_samples_split': 10, 'n_estimators': 200}",4,0.66293,0.81551
4,"{'max_depth': 5, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 200}",5,0.66246,0.81561
5,"{'max_depth': 5, 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 200}",6,0.66244,0.81561


### Identifying Important Features in Random Forests
- see Albon 14.6, page 241-243

#### Calculating Feature Importances

In [51]:
importances = rf_gs_model.best_estimator_.feature_importances_

- sort importances in descending order

In [52]:
indicies = np.argsort(importances)[::-1]

- rearrange feature names so they match the sorted feature importances

In [53]:
names = [X_train_all.columns.tolist()[i] for i in indicies]

In [54]:
important_features = pd.DataFrame(list(zip(indicies, 
                                           names, 
                                           importances)),
                                  columns=['index_number', 
                                           'feature_name', 
                                           'feature_importance']).sort_values(by='feature_importance',
                                                                              ascending=False).reset_index(drop=True)

- top features

In [55]:
important_features.head(10)

,index_number,feature_name,feature_importance
0,0,age,0.13122
1,11,surgdt_month,0.10837
2,2,weightkg,0.08837
3,34,cvd,0.08445
4,4,hct,0.06592
5,30,cancer,0.05619
6,50,diabctrl,0.05016
7,35,cva,0.04937
8,31,pvd,0.04759
9,18,racenativeam,0.02515


- selecting the top 25 features ranked by feature importance

In [56]:
top_features = important_features['feature_name'].values.tolist()[:25]

### Let's see if reducing the number of features improves results

- instantiating a `RandomForestClassifier()` object

In [57]:
rf_clf = RandomForestClassifier(n_estimators=200,
                                class_weight='balanced',
                                random_state=0,
                                n_jobs=-1,
                                verbose=2)

- using only the top features identified above

In [58]:
X_train_top = X_train_all.copy()[top_features]

In [59]:
skf_results = cross_validate(rf_clf, # model
                             X_train_top, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores              

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   33.0s finished


In [60]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.98289,0.98289,0.98304,0.98304,0.98304,0.98298
train_accuracy,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
test_f1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
train_f1,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
test_f1_macro,0.49569,0.49569,0.49572,0.49572,0.49572,0.49571
train_f1_macro,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
test_f1_weighted,0.97441,0.97441,0.97463,0.97463,0.97463,0.97454
train_f1_weighted,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
test_precision,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
train_precision,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


#### Key Takeaways
- Limiting feature set to the top features did not improve results at all with out of the box settings
- Let's try one more scenario using the hyperparameters from our grid search

#### Using `GridSearchCV` Hyperparameters and Top Features

In [61]:
rf_clf = RandomForestClassifier(n_estimators=200,
                                max_depth=2,
                                min_samples_split=2,
                                class_weight='balanced',
                                random_state=0,
                                n_jobs=-1,
                                verbose=2)

In [62]:
skf_results = cross_validate(rf_clf, # model
                             X_train_top, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores              

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   14.9s finished


In [63]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.66121,0.65682,0.65399,0.65853,0.65706,0.65752
train_accuracy,0.66391,0.65331,0.66038,0.66451,0.66184,0.66079
test_f1,0.05544,0.05325,0.04750,0.05351,0.05329,0.05260
train_f1,0.05626,0.05614,0.05800,0.05751,0.05747,0.05708
test_f1_macro,0.42451,0.42184,0.41805,0.42260,0.42195,0.42179
train_f1_macro,0.42590,0.42190,0.42542,0.42672,0.42571,0.42513
test_f1_weighted,0.78096,0.77782,0.77603,0.77916,0.77810,0.77841
train_f1_weighted,0.78298,0.77522,0.78033,0.78336,0.78141,0.78066
test_precision,0.02911,0.02794,0.02492,0.02807,0.02795,0.02760
train_precision,0.02954,0.02943,0.03044,0.03020,0.03017,0.02996


### Final Takeaways
- `RandomForestClassifier` results in similar results to `LogisticRegression` and `STS` model in terms of `roc_auc` scores